Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [23]:
import numpy as np
from google.colab.patches import cv2_imshow
import cv2

# Repeatability
np.random.seed(0)

VFILENAME = "walking.mp4"
HEIGHT = 406
WIDTH = 722

Load video frames from file

In [24]:
def get_frames(filename):
  video=cv2.VideoCapture(filename)
  
  while video.isOpened():
    # returning a flag and frame 
    ret,frame=video.read()
    if ret:
      yield frame
    else:
      break
  
  # Releasing the resources
  video.release()
  yield None

Creating a particle cloud

In [25]:
NUM_PARTICLES = 5000
VEL_RANGE = 0.5

def initialize_particles():
  particles=np.random.rand(NUM_PARTICLES,4)
  particles=particles*np.array((WIDTH,HEIGHT,VEL_RANGE,VEL_RANGE))
  particles[:,:2,:4] -= VEL_RANGE/2.0 

Moving particles according to their velocity state

In [26]:
def apply_velocity(particles):
  partcles[:,0] += particles[:,2]
  partcles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [27]:
def enforce_edges(particles):

  for i in range(NUM_PARTICLES):
    particles[i,0]=max(0,min(WIDTH-1,particles[i,0]))
    particles[i,1]=max(0,min(HEIGHT-1,particles[i,1]))

    return particles

Measure each particle's quality

In [28]:
TARGET_COLOUR=np.array((156,74,38))

def compute_errors(particles, frame):
  errors=np.zeros(NUM_PARTICLES)
  for i in range(NUM_PARTICLES):
    x = int(particles[i,0])
    y = int(particles[i,1])
    pixel_colour=frame[y,x,:]
    errors[i]=np.sum((TARGET_COLOUR-pixel_colour)**2)

    return errors

Assign weights to the particles based on their quality of match

In [29]:
def compute_weights(errors):
    return []

Resample particles according to their weights

In [30]:
def resample(particles, weights):
    return particles, []

Fuzz the particles

In [31]:
def apply_noise(particles):
    return particles

Display the video frames

In [32]:
from google.colab.patches import cv2_imshow
def display(frame, particles, location):

    if len(particles) > 0:
      for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y = int(particles[i,1])
        cv2.circle(frame, (x,y), 1, (0,255,0), 1)
    
    if len(location) > 0:
      cv2.circle(frame , location , 5 , (0,0,255) ,5 )
    
    cv2_imshow(frame)

    if cv2.waitKey(30) == 27:
      if cv2.waitKey(0) == 27:
        return True

    return False

Main routine

In [ ]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()
